# Import statements
Here we import all the required libraries. Please note that in addition to a lot of the standard Python libraries, you will require an installation of PyTorch and a CUDA-compatible GPU to run this code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
torch.set_printoptions(precision=10)

import tpx3_centroiding as tpx3

param_dict = {'spinewidth':3,
              'linewidth':4,
              'ticklength':6,
              'tickwidth':3,
              'ticklabelsize':25,
              'axislabelsize':25,
              'titlesize':30}
import matplotlib
def neaten_plot(neatenme=None, param_dict=param_dict):
    if neatenme is None:
        neatenme = plt.gcf()
    if isinstance(neatenme,matplotlib.figure.Figure):
        for ax in neatenme.axes:
            neaten_plot(ax)
        plt.tight_layout()
    elif isinstance(neatenme,matplotlib.axes.Axes):
        neatenme.tick_params(labelsize=param_dict['ticklabelsize'],length=param_dict['ticklength'],width=param_dict['tickwidth'])
        neatenme.xaxis.get_label().set_fontsize(param_dict['axislabelsize'])
        neatenme.yaxis.get_label().set_fontsize(param_dict['axislabelsize'])
        neatenme.title.set_fontsize(param_dict['titlesize'])
        for axis in ['top','bottom','left','right']:
            neatenme.spines[axis].set_linewidth(param_dict['spinewidth'])
        for line in neatenme.lines:
            line.set_linewidth(param_dict['linewidth'])

In [ ]:
data_array = None
torch.cuda.empty_cache()

for i in range(torch.cuda.device_count()):
    name = torch.cuda.get_device_name(i)
    print(name)
    properties = torch.cuda.get_device_properties(i)
    print(properties)

# Plotting functions
These come in handy later.

In [ ]:
def centroid_mask(cents, mask_ranges=None):
    cols = ['x', 'y', 'tot', 'tof', 'trig', 'param', 'filenum', 'delay']
    if mask_ranges is None:
        mask_ranges = {}
    col_inds = {name: index for index, name in enumerate(cols)}
    mask = np.ones(cents.shape[0], dtype=bool)
    
    for col_name, ranges in mask_ranges.items():
        if col_name not in cols:
            raise ValueError(f"poopy '{col_name}' not found, check again pls.")
        
        col_ind = col_inds[col_name]
        col_mask = np.zeros(cents.shape[0], dtype=bool)
        
        for rmin, rmax in ranges:
            col_mask |= (cents[:, col_ind] >= rmin) & (cents[:, col_ind] <= rmax)
        mask &= col_mask
        
    return cents[mask]

def plot1D(centroids,tt1,tt2):
    param_selec = np.argwhere((centroids[:,3]>tt1) & (centroids[:,3]<tt2)).flatten()
    print(np.shape(param_selec))
    im_xy,xbins,ybins = np.histogram2d(centroids[param_selec,0],centroids[param_selec,1],bins=(np.arange(0,256,xy_binning),np.arange(0,256,xy_binning)))
    im_tot,totbins,tofbins = np.histogram2d(centroids[param_selec,2],centroids[param_selec,3],bins=(np.arange(0,3500,25),np.arange(tt1,tt2,t_binning/2)))

    with np.errstate(divide='ignore'):
        plt.figure(figsize=(10,6))
        plt.title('Tot-tof')
        plt.xlabel('ToF (us)')
        plt.ylabel('ToT (ns)')
        plt.pcolormesh(tofbins,totbins,np.log(im_tot),cmap='inferno')
        plt.xlim(tt1,tt2)
        
def plot2D_xyt(centroids,ttminn,ttmaxx,t_binning,xy_binning,explainy,label2=None,ylim_t = None,
               output=False, TaP = None):
    mask_ranges = {'tof': [(ttminn, ttmaxx)]}
    data = centroid_mask(centroids,mask_ranges)
    print(f"count rate = {data.shape[0]}/{centroids[-1,4]}  = {data.shape[0]/centroids[-1,4]:.2f} per shot")
    im_xy,xbins,ybins = np.histogram2d(data[:,0],data[:,1],bins=(np.arange(0,256,xy_binning),np.arange(0,256,xy_binning)))
    im_xt,xbins,tbins = np.histogram2d(data[:,0],data[:,3],bins=(np.arange(0,256,xy_binning),np.arange(ttminn,ttmaxx,t_binning)))
    im_yt,ybins,tbins = np.histogram2d(data[:,1],data[:,3],bins=(np.arange(0,256,xy_binning),np.arange(ttminn,ttmaxx,t_binning)))
    if TaP is None:    
        countrate2 = data.shape[0]/centroids[-1,4]
    else:
        countrate2 = data.shape[0]/TaP

    with np.errstate(divide='ignore'):
        plt.figure(figsize=(12,4))
        ax1 = plt.subplot(131)
        plt.title('XY')
        plt.pcolormesh(xbins,ybins,np.log(im_xy.T),cmap='inferno')
    
        ax2 = plt.subplot(132)
        plt.title('XT')
        plt.pcolormesh(xbins,tbins,np.log(im_xt.T),cmap='inferno')
        if ylim_t:
            plt.ylim(ylim_t)
    
        ax3 = plt.subplot(133)
        plt.title('YT')
        plt.pcolormesh(ybins,tbins,np.log(im_yt.T),cmap='inferno')
        if ylim_t:
            plt.ylim(ylim_t)

    if label2 is None:
        plt.suptitle(f'{explainy}, count rate={countrate2:.4f} /shot')
    else:
        plt.suptitle(f'{explainy}, {label2}, count rate={countrate2:.4f} /shot')
    plt.show()
    
    if output:
        return im_xy,xbins,ybins

def plot_tottof(centroids,ttminn,ttmaxx,t_binning,mzfit=None,mzs=[]):
    mask_ranges = {'tof': [(ttminn, ttmaxx)]}
    data = centroid_mask(centroids,mask_ranges)
    totmax = 3500
    im_tot,totbins,tofbins = np.histogram2d(data[:,2],data[:,3],bins=(np.arange(0,totmax,25),np.arange(tt1,tt2,t_binning/2)))

    with np.errstate(divide='ignore'):
        plt.figure(figsize=(10,6))
        plt.title('Tot-tof')
        plt.xlabel('ToF (us)')
        plt.ylabel('ToT (ns)')
        plt.pcolormesh(tofbins,totbins,np.log(im_tot),cmap='inferno')
        if mzfit is not None:
            tofs = mz_to_tof(mzs,mzfit)
            masky = (tofs>ttminn) & (tofs<ttmaxx)
            tofs2 = tofs[masky]
            plt.vlines([tofs2],0,totmax-100,'g')
        plt.xlim(tt1,tt2)

def plotXY_large(centroids,ttminn,ttmaxx,t_binning,circles=[],xlim=None,ylim=None):
    mask_ranges = {'tof': [(ttminn, ttmaxx)]}
    data = centroid_mask(centroids,mask_ranges)
    im_xy,xbins,ybins = np.histogram2d(data[:,0],data[:,1],bins=(np.arange(0,256,xy_binning),np.arange(0,256,xy_binning)))
    count_rate = data.shape[0]/centroids[-1,4]

    with np.errstate(divide='ignore'):
        plt.figure(figsize=(10,8))
        plt.title(explainy + f', count rate={count_rate:.4f} e-/shot, XY')
        plt.pcolormesh(xbins,ybins,np.log(im_xy.T),cmap='inferno',vmin=0)#,vmin=3,vmax=8)
        for c in circles:
            plt.gca().add_patch(c)
        plt.colorbar()
        if xlim:
            plt.xlim(xlim)
        if ylim:
            plt.ylim(ylim)
        plt.show()

# Centroiding for individual text files
Here we read in a `.txt` file of TimePix3 data and centroid it. The output will be a list of centroid values that correspond to individual hits on our detector.

In [ ]:
filenames = ['test_tpx3_file.txt']

batch_size = 10
skiprows = 0 # should usually be 0

torch.cuda.empty_cache()
for filename in filenames:
    print(filename)
    read_line_num = None
    start_time = time.time()
    centroids = tpx3.read_file_batched(filename, batch_size=batch_size,
                                  read_line_num=read_line_num, skiprows=skiprows)
    end_time = time.time()
    np.save(f'{filename[:-4]}_centroids.npy',centroids)
    print(f'{len(centroids)} centroids total, time to execute : {end_time-start_time:.5f} s')

# Some basic plotting
Here we plot the X-Y, X-ToF, Y-ToF, and ToT-ToF distributions for the electrons and ions in our short dataset. You can change which regions of ToF you are plotting by adjusting the `ttmin` and `ttmax` variables.

In [ ]:
explainy = 'argon data'

ttminn = 3
ttmaxx = 4
tcutlab1 = 'electrons'

ttmin2 = 4
ttmax2 = 8
tcutlab2 = 'ions'

ttmin3 = 5.75
ttmax3 = 6.2
tcutlab3 = 'water ions'
tot_min = 300
xy_binning = 1
t_binning = 0.00156233478

plotXY_large(centroids,ttminn,ttmaxx,t_binning)
plot2D_xyt(centroids,ttminn,ttmaxx,t_binning,xy_binning,explainy,tcutlab1)

plot2D_xyt(centroids,ttmin2,ttmax2,t_binning,xy_binning,explainy,tcutlab2)
plot2D_xyt(centroids,ttmin3,ttmax3,t_binning,xy_binning,explainy,tcutlab3)

tt1 = 3
tt2 = 8
plot_tottof(centroids,tt1,tt2,t_binning)
